In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p /content/drive/MyDrive/PURE

In [ ]:
!git clone https://github.com/mika-gjl/PURE.git /content/drive/MyDrive/PURE/PURE

Cloning into '/content/drive/MyDrive/PURE/PURE'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 95 (delta 45), reused 84 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 2.48 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
!sudo apt update
!sudo apt install -y python3.10 python3.10-distutils python3.10-dev
!curl -sS https://bootstrap.pypa.io/get-pip.py | sudo python3.10


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,780 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,190 kB]
Fetched 12.2 MB in 4s (3,112 kB/s)
Reading pa

In [ ]:
# 安装 tokenizers 需要 rustc
!sudo apt install -y rustc cargo

# 使用 python3.10 安装指定版本
!/usr/bin/python3.10 -m pip install -U pip setuptools wheel
!/usr/bin/python3.10 -m pip install tqdm==4.67.1
!/usr/bin/python3.10 -m pip install torch==1.12.1 --index-url https://download.pytorch.org/whl/cu113
!/usr/bin/python3.10 -m pip install allennlp==2.10.1
!/usr/bin/python3.10 -m pip install transformers==4.20.1 tokenizers==0.12.1


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
rustc is already the newest version (1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04).
cargo is already the newest version (1.75.0+dfsg0ubuntu1~bpo0-0ubuntu0.22.04).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
Looking in indexes: https://download.pytorch.org/whl/cu113


In [ ]:
!/usr/bin/python3.10 -c "import sys; print('Python version:', sys.version)"
!/usr/bin/python3.10 -c "import torch; print('torch:', torch.__version__)"
!/usr/bin/python3.10 -c \"from transformers import AutoTokenizer; print('transformers imported OK')\"


Python version: 3.10.12 (main, Aug 15 2025, 14:32:43) [GCC 11.4.0]
torch: 1.12.1+cu113
/bin/bash: -c: line 1: syntax error near unexpected token `'transformers imported OK''
/bin/bash: -c: line 1: `/usr/bin/python3.10 -c \"from transformers import AutoTokenizer; print('transformers imported OK')\"'


In [ ]:
!/usr/bin/python3.10 -m pip install numpy==1.23.5


In [ ]:
code = '''
from transformers import CamembertTokenizer, CamembertModel

save_path = "/content/drive/MyDrive/PURE/PURE/camembert-base"

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

print("✅ CamemBERT 模型与 tokenizer 已成功保存到:", save_path)
'''

with open("download_camembert_310.py", "w") as f:
    f.write(code)


In [ ]:
!/usr/bin/python3.10 download_camembert_310.py


Downloading: 100% 792k/792k [00:00<00:00, 6.40MB/s]
Downloading: 100% 25.0/25.0 [00:00<00:00, 165kB/s]
Downloading: 100% 508/508 [00:00<00:00, 3.38MB/s]
Downloading: 100% 424M/424M [00:04<00:00, 93.1MB/s]
Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
✅ CamemBERT 模型与 tokenizer 已成功保存到: /content/drive/MyDrive/PURE/PURE/camembert-base


In [ ]:
/usr/bin/python3.10 run_entity.py \
  --task custom \
  --data_dir "/content/drive/MyDrive/PURE/PURE/data" \
  --output_dir "/content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300" \
  --model camembert-base \
  --bert_model_dir "/content/drive/MyDrive/PURE/PURE/camembert-base" \
  --context_window 0 \
  --max_span_length 8 \
  --train_batch_size 16 \
  --learning_rate 1e-5 \
  --task_learning_rate 5e-4 \
  --num_epoch 100 \
  --warmup_proportion 0.1 \
  --do_train --do_eval --eval_test \
  --train_shuffle \
  --seed 42 \
  --pretokenized


SyntaxError: invalid syntax (ipython-input-3640340052.py, line 1)

In [ ]:
/usr/bin/python3.10 run_relation.py \
  --task custom \
  --do_train \
  --train_file "/content/drive/MyDrive/PURE/PURE/data/train_pure.jsonl" \
  --do_eval \
  --eval_test \
  --model camembert-base \
  --train_batch_size 16 \
  --eval_batch_size 16 \
  --learning_rate 2e-5 \
  --num_train_epochs 5 \
  --context_window 0 \
  --max_seq_length 228 \
  --entity_output_dir "/content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300" \
  --output_dir "/content/drive/MyDrive/PURE/PURE/outputs/relation_model"


In [ ]:
/usr/bin/python3.10 run_relation.py \
  --task custom \
  --do_train \
  --train_file "/content/drive/MyDrive/PURE/PURE/data/train_pure.jsonl" \
  --do_eval \
  --eval_test \
  --model camembert-base \
  --train_batch_size 16 \
  --eval_batch_size 16 \
  --learning_rate 2e-5 \
  --num_train_epochs 5 \
  --context_window 0 \
  --max_seq_length 228 \
  --entity_output_dir "/content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300" \
  --output_dir "/content/drive/MyDrive/PURE/PURE/outputs/relation_model" \
  --prediction_file predictions.json \
  --entity_predictions_test /content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300/ent_pred_test.json


In [ ]:
/usr/bin/python3.10 run_relation.py \
  --task custom \
  --do_eval \
  --eval_test \
  --model camembert-base \
  --eval_batch_size 16 \
  --context_window 0 \
  --max_seq_length 228 \
  --entity_output_dir "/content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300" \
  --output_dir "/content/drive/MyDrive/PURE/PURE/outputs/relation_model" \
  --prediction_file predictions.json \
  --entity_predictions_test /content/drive/MyDrive/PURE/PURE/outputs/camembert-base_ctx300/ent_pred_test.json


In [ ]:
!nvidia-smi


Fri Aug 22 14:56:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             72W /   70W |    6148MiB /  15360MiB |     98%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----